### Install Libraries

In [2]:
!pip install sklearn --quiet
!pip install nltk --quiet
!pip install keras --quiet
!pip install seaborn --quiet
!pip install tensorflow --quiet
!pip install biobert-embedding --quiet

In [3]:
import io
#from Bio import Entrez, Medline
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import re
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm
#tqdm.pandas(desc="progress-bar")
#from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
#import gensim
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier 
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,make_scorer,precision_score,recall_score,roc_auc_score,f1_score
from sklearn.model_selection import GridSearchCV
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#from gensim.models.doc2vec import TaggedDocument
import re
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request, urllib.error, urllib.parse
import json
import os
from pprint import pprint
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
#import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
#import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
#from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')
from bs4 import BeautifulSoup
#from rouge import Rouge
from sklearn.decomposition import PCA

import warnings
warnings.simplefilter("ignore", UserWarning)
import getpass
import glob

from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation 

from biobert_embedding.embedding import BiobertEmbedding

2022-10-10 13:02:33.578979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 13:02:33.768859: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oss/hadoop/lib/native/
2022-10-10 13:02:33.768897: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-10 13:02:33.806057: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-10 13:02:34.8

#### 1. Read input files for Covid 19-Metadata / Brain Aneurysm

In [4]:
# Covid 19 raw data
df_covid19_raw=pd.read_csv('./data/metadata.csv')
# Covid 19 PICO classified data
##df_covid19_PICO=pd.read_csv('./data/df_covid19_PICO.csv')
# Covid 19 PICO data with biobert encoder
df_covid19_PICO_biobert=pd.read_csv('./data/df_covid19_PICO_biobert_encoded_combined.csv')

df_covid19_PICO_biobert.dropna(subset=['sent'],inplace=True)
df_covid19_PICO_biobert.dropna(subset=['sent_embedding'],inplace=True)
df_covid19_PICO_biobert['sent_embedding_array']=df_covid19_PICO_biobert['sent_embedding'].apply(lambda x:np.array(eval(x.replace('\n','').replace('tensor(','').replace(')',''))))

 # CB Data

# CB data with bioBERT encoder
df_BA_APIRO_biobert=pd.read_csv('./data/df_APIRO_Dataset_biobert_encoded_combined.csv')
# remove nan and null
df_BA_APIRO_biobert.dropna(subset=['sent'],inplace=True)
df_BA_APIRO_biobert.dropna(subset=['sent_embedding'],inplace=True)
df_BA_APIRO_biobert['sent_embedding_array']=df_BA_APIRO_biobert['sent_embedding'].apply(lambda x:np.array(eval(x.replace('\n','').replace('tensor(','').replace(')',''))))

/opt/oss/conda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_BA_APIRO_biobert['aimoprc_category'].value_counts()

#### 2. Create Cluster to indetify different topics

In [ ]:
df_covid19_clustering=df_covid19_raw[['pmcid','pubmed_id','abstract']]
df_covid19_clustering.dropna(subset=['abstract'],inplace=True)
df_covid19_clustering = df_covid19_clustering.reset_index(drop=True)
df_covid19_clustering.rename(columns={'abstract':'sent'},inplace=True)

In [ ]:
df_covid19_clustering.info()

In [9]:
def clean_text(text):
    '''
    @author Fakhare Alam
    '''
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords.words('english'))
    
    text = text.lower()

    text = REPLACE_BY_SPACE_RE.sub(' ', text) 

    text = BAD_SYMBOLS_RE.sub('', text) 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

## Keras Vector Generation

In [10]:
def keras_token_generator(df):
    '''
    @author : Fakhare Alam
    '''
 
    #The maximum number of words to be used
    MAX_NB_WORDS = 50000
    # Max number of words in each sentence
    MAX_SEQUENCE_LENGTH = 250
    # Embeddng dimension
    EMBEDDING_DIM = 100
    df['sent'] = df['sent'].apply(clean_text)
    df['sent'] =df['sent'].str.replace('\d+', '')
    tokenizer_keras = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer_keras.fit_on_texts(df['sent'].values)
    word_index_keras = tokenizer_keras.word_index
    print('Found %s unique tokens.' % len(word_index_keras))

    X_keras = tokenizer_keras.texts_to_sequences(df['sent'].values)
    X_keras = pad_sequences(X_keras, maxlen=MAX_SEQUENCE_LENGTH)
    print('Shape of data tensor:',X_keras.shape)
    return X_keras
    

##### Keras data tokens

In [ ]:
# raw Covid 19
X_covid=keras_token_generator(df_covid19_clustering)
# Covid 19 with PICO
X_covid_PICO=keras_token_generator(df_covid19_PICO_biobert)#keras_token_generator(df_covid19_PICO)
# CB
X_CB=keras_token_generator(df_BA_APIRO_biobert)#keras_token_generator(df_BA_APIRO)

### Principal Components Analsysis

In [ ]:
def create_PCA(X):
    '''
    '''
    pca_2=PCA(n_components=2)
    # Scaled Data
    pca_data=pca_2.fit_transform(X)

    #Pickle the PCA model
    #pickle.dump(pca_2_covid, open('pca', 'wb'))

    df_pca_data=pd.DataFrame(data=pca_data,columns=['first','second'])
    return df_pca_data,pca_data
    

#### Covid Data

In [ ]:
df_covid_pca_data,pca_data_covid=create_PCA(X_covid)
df_covid_pca_data.head()

#### CB Data

In [ ]:
df_BA_APIRO=pd.read_csv('./data/APIRO_Dataset.csv').drop(columns='Unnamed: 0',axis=1)
df_CB_pca_data,pca_data_CB=create_PCA(keras_token_generator(df_BA_APIRO))
df_CB_pca_data.head()

##### PCA performence

# Eigen Values and Eigen vector
print('Eigen Values',pca_2_CB.explained_variance_)
#print('Eigen Vectors',pca_2.components_)
print('\n')
print('Explained Variance Ratio',pca_2_CB.explained_variance_ratio_)


plt.figure(figsize=(10,8))
plt.scatter(pca_data_CB[:,0],pca_data_CB[:,1])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('New projection using PCA with 2 dimensions',fontsize=15)


#### K means clustering to optimize number of topics

In [16]:
def k_means_iteration(pca_data):
    
    '''
    @author - Fakhare Alam
    '''
    inertias = [] 
    mapping_inertia = {} 
    K = range(2,10) 
    mapping_inertia['#clusetrs']='SSE'
    for k in K: 
        #Building and fitting the model 
        kmeanModel = KMeans(n_clusters=k) 
        kmeanModel.fit(pca_data)        
        inertias.append(kmeanModel.inertia_) 
        mapping_inertia[k] = kmeanModel.inertia_ 

    for key,val in mapping_inertia.items(): 
        print(str(key)+' : '+str(val))
    plt.plot(K, inertias, 'bx-') 
    plt.xlabel('k (number of clusters)') 
    plt.ylabel('Sum of Squared Distances') 
    plt.title('Covid 19 dataset clustering') 
    plt.show() 

##### Covid 19

In [ ]:
k_means_iteration(pca_data_covid)

##### CB

In [ ]:
k_means_iteration(pca_data_CB)

### Map on optimized number of clustering and map it back to original sentences.


In [19]:
def do_Kmeans(X, nclust=2):
    '''
    @ Author - Fakhare Alam
    This method will gemerate cluster based on
    givennumber of clusters
    '''
    model = KMeans(n_clusters=nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    
    # Pickle the Kmeans Model
    #pickle.dump(model, open('Kmeans', 'wb'))
    return (clust_labels, cent)


##### Covid 19

In [ ]:
# Label Cluster
knn_clust_labels, cent = do_Kmeans(pca_data_covid, 4)

#Combine label with scaled (PCA) data and raw data
df_covid19_clustering[['First PCA Component','Second PCA Component']]=df_covid_pca_data[['first','second']]
df_covid19_clustering['knn_clust_labels']=knn_clust_labels

# Check Value Counts of all the label
df_covid19_clustering['knn_clust_labels'].value_counts()

plt.figure(figsize=(8,6))
sns.scatterplot(x="First PCA Component", y="Second PCA Component", hue="knn_clust_labels",data=df_covid19_clustering,palette="Set2",legend='full').set(title='Covid 19 cluster visulization using PCA components')

#### CB

In [ ]:
# Label Cluster
knn_clust_labels, cent = do_Kmeans(pca_data_CB, 4)
print('Cluster Labels',knn_clust_labels)
print('Centroid of cluster',cent)

#Combine label with scaled (PCA) data and raw data
df_BA_APIRO[['First PCA Component','Second PCA Component']]=df_CB_pca_data[['first','second']]
df_BA_APIRO['knn_clust_labels']=knn_clust_labels

# Check Value Counts of all the label
df_BA_APIRO['knn_clust_labels'].value_counts()

plt.figure(figsize=(8,6))
sns.scatterplot(x="First PCA Component", y="Second PCA Component", hue="knn_clust_labels",data=df_BA_APIRO,palette="Set2",legend='full').set(title='Cerebral Aneurysm data cluster visulization using PCA omponents')

#### Topic Modelling - to determine theme in identifies cluster

#### Generic multi class classifier

In [ ]:
def multiclass_classifier(X_train,Y_train,X_test,Y_test,model,classifier):
    '''
    @author : Fakhare Alam
    '''
    print(classifier)
    print('\n')
    model.fit(X_train, Y_train)
    Y_pred=model.predict(X_test)
    print('Confusion Matrix\n')
    print(confusion_matrix(Y_test,Y_pred))
    print('Classification Report\n')
    print(classification_report(Y_test,Y_pred))

### CB Dataset -Keras embedding

In [ ]:
Y_CB = df_BA_APIRO_biobert['aimoprc_category']
Y_L_CB = pd.get_dummies(df_BA_APIRO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_L_CB.shape)
X_CB_train, X_CB_test, Y_CB_train, Y_CB_test = train_test_split(X_CB,Y_CB, test_size = 0.10, random_state = 42)
X_CB_train_LSTM, X_CB_test_LSTM,Y_CB_train_LSTM, Y_CB_test_LSTM = train_test_split(X_CB,Y_L_CB, test_size = 0.10, random_state = 42)
print('train dataset shape\n')
print(X_CB_train.shape,Y_CB_train.shape)
print('test dataset shape\n')
print(X_CB_test.shape,Y_CB_test.shape)

# Base Model 
logreg_CB=LogisticRegression()
knn_CB = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
adaboost_CB = AdaBoostClassifier(random_state=1)
Gboost_CB = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
mlp_CB = MLPClassifier(hidden_layer_sizes=(100, 70, 30,20,10), max_iter=1000)

run_models=[logreg_CB,knn_CB,adaboost_CB,Gboost_CB,mlp_CB]

for model in run_models:
    multiclass_classifier(X_CB_train,Y_CB_train,X_CB_test,Y_CB_test,model,str(model))

# LSTM Model
#The maximum number of words to be used
MAX_NB_WORDS = 50000
# Max number of words in each sentence
MAX_SEQUENCE_LENGTH = 250
# Embeddng dimension
EMBEDDING_DIM = 100
lstm_model_CB = Sequential()
lstm_model_CB.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_CB.shape[1]))
lstm_model_CB.add(SpatialDropout1D(0.2))
lstm_model_CB.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model_CB.add(Dense(5, activation='softmax'))
lstm_model_CB.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model_CB.summary())

###

epochs = 5
batch_size = 64
history = lstm_model_CB.fit(X_CB_train_LSTM, Y_CB_train_LSTM, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

plt.title('CB Dataset Bi-LSTM Keras Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


plt.title('CB Dataset Bi-LSTM Keras Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

Y_CB_pred = np.argmax(lstm_model_CB.predict(X_CB_test_LSTM),axis=1)
print(confusion_matrix(np.argmax(Y_CB_test_LSTM,axis=1),Y_CB_pred))
print(classification_report(np.argmax(Y_CB_test_LSTM,axis=1),Y_CB_pred))


### CB Dataset-bioBERT embedding

In [ ]:
X_CB_biobert=np.vstack(df_BA_APIRO_biobert['sent_embedding_array'])
X_CB_biobert=X_CB_biobert[:, :50]
Y_CB_biobert = df_BA_APIRO_biobert['aimoprc_category']
Y_L_CB_biobert = pd.get_dummies(df_BA_APIRO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_L_CB_biobert.shape)
X_CB_train_biobert, X_CB_test_biobert, Y_CB_train_biobert, Y_CB_test_biobert = train_test_split(X_CB_biobert,Y_CB_biobert, test_size = 0.10, random_state = 42)
X_CB_train_LSTM_biobert, X_CB_test_LSTM_biobert,Y_CB_train_LSTM_biobert, Y_CB_test_LSTM_biobert = train_test_split(X_CB_biobert,Y_L_CB_biobert, test_size = 0.10, random_state = 42)
print('train dataset shape')
print(X_CB_train_biobert.shape,Y_CB_train_biobert.shape)
print('test dataset shape')
print(X_CB_test_biobert.shape,Y_CB_test_biobert.shape)

# Base Model
logreg_CB_biobert=LogisticRegression()
knn_CB_biobert = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
adaboost_CB_biobert = AdaBoostClassifier(random_state=1)
Gboost_CB_biobert = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
mlp_CB_biobert = MLPClassifier(hidden_layer_sizes=(100, 70, 30,20,10), max_iter=1000)

run_models=[logreg_CB_biobert,knn_CB_biobert,adaboost_CB_biobert,Gboost_CB_biobert,mlp_CB_biobert]

for model in run_models:
    multiclass_classifier(X_CB_train_biobert,Y_CB_train_biobert,X_CB_test_biobert,Y_CB_test_biobert,model,str(model))
    
# LSTM Model
X_CB_biobert=np.vstack(df_BA_APIRO_biobert['sent_embedding_array'])
X_CB_biobert=X_CB_biobert[:, :300]
#X_CB_biobert=abs(X_CB_biobert)
Y_CB_biobert = df_BA_APIRO_biobert['aimoprc_category']
Y_L_CB_biobert = pd.get_dummies(df_BA_APIRO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_L_CB_biobert.shape)
X_CB_train_biobert, X_CB_test_biobert, Y_CB_train_biobert, Y_CB_test_biobert = train_test_split(X_CB_biobert,Y_CB_biobert, test_size = 0.10, random_state = 42)
X_CB_train_LSTM_biobert, X_CB_test_LSTM_biobert,Y_CB_train_LSTM_biobert, Y_CB_test_LSTM_biobert = train_test_split(X_CB_biobert,Y_L_CB_biobert, test_size = 0.10, random_state = 42)
print('train dataset shape')
print(X_CB_train_biobert.shape,Y_CB_train_biobert.shape)
print('test dataset shape')
print(X_CB_test_biobert.shape,Y_CB_test_biobert.shape)




lstm_model_CB_biobert = Sequential()
input_vector=300
lstm_model_CB_biobert.add(LSTM(input_vector, dropout=0.4, recurrent_dropout=0.4))
lstm_model_CB_biobert.add(Dense(5, activation='softmax'))
lstm_model_CB_biobert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model_CB_biobert.build((None, input_vector,1))
print(lstm_model_CB_biobert.summary())

###

epochs = 5
batch_size = 64
history = lstm_model_CB_biobert.fit(X_CB_train_LSTM_biobert, 
                                    Y_CB_train_LSTM_biobert, 
                                    epochs=epochs, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
                                   )

plt.title('CB Dataset Bi-LSTM bioBERT Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


plt.title('CB Dataset Bi-LSTM bioBERT Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

Y_CB_pred_biobert = np.argmax(lstm_model_CB_biobert.predict(X_CB_test_LSTM_biobert),axis=1)
print(confusion_matrix(np.argmax(Y_CB_test_LSTM_biobert,axis=1),Y_CB_pred_biobert))
print(classification_report(np.argmax(Y_CB_test_LSTM_biobert,axis=1),Y_CB_pred_biobert))

### Covid Dataset -Keras Embedding

In [ ]:
Y_covid_PICO= df_covid19_PICO_biobert['aimoprc_category']
Y_L_covid_PICO = pd.get_dummies(df_covid19_PICO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_covid_PICO.shape)

X_covid_PICO_train, X_covid_PICO_test, Y_covid_PICO_train, Y_covid_PICO_test = train_test_split(X_covid_PICO,Y_covid_PICO, test_size = 0.10, random_state = 42)

X_covid_PICO_train_LSTM, X_covid_PICO_test_LSTM,Y_covid_PICO_train_LSTM, Y_covid_PICO_test_LSTM = train_test_split(X_covid_PICO,Y_L_covid_PICO, test_size = 0.10, random_state = 42)

print(X_covid_PICO_train.shape,Y_covid_PICO_train.shape)
print(X_covid_PICO_test.shape,Y_covid_PICO_test.shape)

# Base Model
logreg_covid=LogisticRegression()
knn_covid = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
adaboost_covid = AdaBoostClassifier(random_state=1)
Gboost_covid = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
mlp_covid = MLPClassifier(hidden_layer_sizes=(100, 70, 30,20,10), max_iter=1000)

run_models=[logreg_covid,knn_covid,adaboost_covid,Gboost_covid,mlp_covid]

for model in run_models:
    multiclass_classifier(X_covid_PICO_train,Y_covid_PICO_train,X_covid_PICO_test,Y_covid_PICO_test,model,str(model))
    
# LSTM Model
lstm_model_covid = Sequential()
lstm_model_covid.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_CB.shape[1]))
lstm_model_covid.add(SpatialDropout1D(0.2))
lstm_model_covid.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model_covid.add(Dense(5, activation='softmax'))
lstm_model_covid.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model_covid.summary())

###

epochs = 5
batch_size = 64
history = lstm_model_covid.fit(X_covid_PICO_train_LSTM, Y_covid_PICO_train_LSTM, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

Y_covid_PICO_pred = np.argmax(lstm_model_covid.predict(X_covid_PICO_test_LSTM),axis=1)
print(confusion_matrix(np.argmax(Y_covid_PICO_test_LSTM,axis=1),Y_covid_PICO_pred))
print(classification_report(np.argmax(Y_covid_PICO_test_LSTM,axis=1),Y_covid_PICO_pred))

### Covid Dataset-BioBERT Embedding

In [ ]:
X_covid_PICO_biobert=np.vstack(df_covid19_PICO_biobert['sent_embedding_array'])
X_covid_PICO_biobert=X_covid_PICO_biobert[:, :50]
Y_covid_PICO_biobert= df_covid19_PICO_biobert['aimoprc_category']
Y_L_covid_PICO_biobert = pd.get_dummies(df_covid19_PICO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_covid_PICO_biobert.shape)

X_covid_PICO_train_biobert, X_covid_PICO_test_biobert, Y_covid_PICO_train_biobert, Y_covid_PICO_test_biobert = train_test_split(X_covid_PICO_biobert,Y_covid_PICO_biobert, test_size = 0.10, random_state = 42)

X_covid_PICO_train_LSTM_biobert, X_covid_PICO_test_LSTM_biobert,Y_covid_PICO_train_LSTM_biobert, Y_covid_PICO_test_LSTM_biobert = train_test_split(X_covid_PICO_biobert,Y_L_covid_PICO_biobert, test_size = 0.10, random_state = 42)

print(X_covid_PICO_train_biobert.shape,Y_covid_PICO_train_biobert.shape)
print(X_covid_PICO_test_biobert.shape,Y_covid_PICO_test_biobert.shape)

# Base Model
logreg_covid_biobert=LogisticRegression()
knn_covid_biobert = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
adaboost_covid_biobert = AdaBoostClassifier(random_state=1)
Gboost_covid_biobert = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
mlp_covid_biobert = MLPClassifier(hidden_layer_sizes=(100, 70, 30,20,10), max_iter=1000)

run_models=[logreg_covid_biobert,knn_covid_biobert,adaboost_covid_biobert,Gboost_covid_biobert,mlp_covid_biobert]

for model in run_models:
    multiclass_classifier(X_covid_PICO_train_biobert,Y_covid_PICO_train_biobert,X_covid_PICO_test_biobert,Y_covid_PICO_test_biobert,model,str(model))
# LSTM Model



In [ ]:
# LSTM Model
X_covid_PICO_biobert=np.vstack(df_covid19_PICO_biobert['sent_embedding_array'])
#X_covid_PICO_biobert=X_covid_PICO_biobert[:, :500]
Y_covid_PICO_biobert= df_covid19_PICO_biobert['aimoprc_category']
Y_L_covid_PICO_biobert = pd.get_dummies(df_covid19_PICO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_covid_PICO_biobert.shape)


X_covid_PICO_train_LSTM_biobert, X_covid_PICO_test_LSTM_biobert,Y_covid_PICO_train_LSTM_biobert, Y_covid_PICO_test_LSTM_biobert = train_test_split(X_covid_PICO_biobert,Y_L_covid_PICO_biobert, test_size = 0.10, random_state = 42)
print('train dataset shape')
print(X_covid_PICO_train_LSTM_biobert.shape,Y_covid_PICO_train_LSTM_biobert.shape)
print('test dataset shape')
print(X_covid_PICO_test_LSTM_biobert.shape,Y_covid_PICO_test_LSTM_biobert.shape)




lstm_model_covid_biobert = Sequential()
input_vector=768
lstm_model_covid_biobert.add(LSTM(input_vector, dropout=0.2, recurrent_dropout=0.2))
lstm_model_covid_biobert.add(Dense(5, activation='softmax'))
lstm_model_covid_biobert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model_covid_biobert.build((None, input_vector,1))
print(lstm_model_covid_biobert.summary())

###

epochs = 5
batch_size = 64
history = lstm_model_covid_biobert.fit(X_covid_PICO_train_LSTM_biobert, 
                                    Y_covid_PICO_train_LSTM_biobert, 
                                    epochs=epochs, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
                                   )

plt.title('CB Dataset Bi-LSTM bioBERT Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


plt.title('CB Dataset Bi-LSTM bioBERT Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

Y_covid_PICO_pred_biobert = np.argmax(lstm_model_covid_biobert.predict(X_covid_PICO_test_LSTM_biobert),axis=1)
print(confusion_matrix(np.argmax(Y_covid_PICO_test_LSTM_biobert,axis=1),Y_covid_PICO_pred_biobert))
print(classification_report(np.argmax(Y_covid_PICO_test_LSTM_biobert,axis=1),Y_covid_PICO_pred_biobert))

In [ ]:
# LSTM Model
X_covid_PICO_biobert=np.vstack(df_covid19_PICO_biobert['sent_embedding_array'])
#X_covid_PICO_biobert=X_covid_PICO_biobert[:, :500]
Y_covid_PICO_biobert= df_covid19_PICO_biobert['aimoprc_category']
Y_L_covid_PICO_biobert = pd.get_dummies(df_covid19_PICO_biobert['aimoprc_category']).values
print('Shape of label tensor:', Y_covid_PICO_biobert.shape)


X_covid_PICO_train_LSTM_biobert, X_covid_PICO_test_LSTM_biobert,Y_covid_PICO_train_LSTM_biobert, Y_covid_PICO_test_LSTM_biobert = train_test_split(X_covid_PICO_biobert,Y_L_covid_PICO_biobert, test_size = 0.10, random_state = 42)
print('train dataset shape')
print(X_covid_PICO_train_LSTM_biobert.shape,Y_covid_PICO_train_LSTM_biobert.shape)
print('test dataset shape')
print(X_covid_PICO_test_LSTM_biobert.shape,Y_covid_PICO_test_LSTM_biobert.shape)




lstm_model_covid_biobert = Sequential()
input_vector=768
lstm_model_covid_biobert.add(LSTM(input_vector, dropout=0.2, recurrent_dropout=0.2))
lstm_model_covid_biobert.add(Dense(5, activation='softmax'))
lstm_model_covid_biobert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model_covid_biobert.build((None, input_vector,1))
print(lstm_model_covid_biobert.summary())

###

epochs = 5
batch_size = 64
history = lstm_model_covid_biobert.fit(X_covid_PICO_train_LSTM_biobert, 
                                    Y_covid_PICO_train_LSTM_biobert, 
                                    epochs=epochs, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
                                   )

plt.title('CB Dataset Bi-LSTM bioBERT Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


plt.title('CB Dataset Bi-LSTM bioBERT Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

Y_covid_PICO_pred_biobert = np.argmax(lstm_model_covid_biobert.predict(X_covid_PICO_test_LSTM_biobert),axis=1)
print(confusion_matrix(np.argmax(Y_covid_PICO_test_LSTM_biobert,axis=1),Y_covid_PICO_pred_biobert))
print(classification_report(np.argmax(Y_covid_PICO_test_LSTM_biobert,axis=1),Y_covid_PICO_pred_biobert))